In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext lab_black
# ignore notebook errors
import warnings

warnings.filterwarnings("ignore")

Dataset from:  
https://www.kaggle.com/datasets/subhamjain/temperature-of-all-countries-19952020

In [ ]:
raw = pd.read_csv("../data/city_temperature.csv")

In [ ]:
raw.head()

In [ ]:
raw.info()

In [ ]:
# Add column with temperature in celsius
temperature = raw.copy()
temperature["AvgTemperatureC"] = ((temperature["AvgTemperature"] - 32) * 5 / 9).round(1)
temperature.rename(columns={"AvgTemperature": "AvgTemperatureF"}, inplace=True)
del raw

In [ ]:
# format column names to be only lowercase
temperature.columns = temperature.columns.str.lower()

In [ ]:
temperature.head()

In [ ]:
# Check a couple to make sure they are correct
temperature[temperature["city"] == "Madrid"].groupby("month")[
    "avgtemperaturec"
].mean().plot(kind="bar")
plt.title("Average Temperature in Madrid by Month")
plt.show()

In [ ]:
temperature[temperature["city"] == "Stockholm"].groupby("month")[
    "avgtemperaturec"
].mean().plot(kind="bar")
plt.title("Average Temperature in Madrid by Month")
plt.show()

## Evaluation 
Everything seems correct  
Lets move it to a database sqlite since it comes with python

In [ ]:
import sqlite3

In [ ]:
with sqlite3.connect("../data/my_database.db") as conn:
    temperature.to_sql("temperature", conn, if_exists="replace", index=False)

In [ ]:
# lets try it out
with sqlite3.connect("../data/my_database.db") as conn:
    df = pd.read_sql_query("SELECT DISTINCT COUNTRY FROM temperature;", conn)

df["country"].to_list()

In [ ]:
# save a list of the countries as a json
import json

with open("../data/countries.json", "w") as f:
    json.dump(df["country"].to_list(), f)

In [ ]:
def get_schema(database_path, table_name=None):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # If a specific table name is provided
    if table_name:
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        for column in columns:
            print(column)
    # If no specific table name is provided, retrieve schema for all tables
    else:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        for table in tables:
            table_name = table[0]
            print(f"Schema for {table_name}:")
            cursor.execute(f"PRAGMA table_info({table_name})")
            columns = cursor.fetchall()
            for column in columns:
                print(column)
            print("------")

    # Close the connection
    conn.close()


# Usage
database_path = "../data/my_database.db"
get_schema(database_path)  # Get schema for all tables
# g